In [ ]:
#from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import matplotlib.patheffects as path_effects
import sys as s
import os 
from cartopy import config
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
dd =pd.date_range(start='1/1/2020', end='31/12/2020')

In [ ]:
dd_array = np.array(dd,dtype= 'str')

In [ ]:
datessss = []
for i in np.arange(0,len(dd_array)):
    year  = dd_array[i][0:4]
    month = dd_array[i][5:7]
    day   = dd_array[i][8:10]
    one = year+month+day
    datessss.append(one)
dates = np.array(datessss,dtype='str')

In [ ]:
dates.shape

In [ ]:
DUSMASS25_con = []
OCSMASS_con   = []
BCSMASS_con   = []
SSSMASS25_con = []
SO4SMASS_con = []
for i in dates:
    try:
        data = Dataset(r'J:\MERRA-2 Aerosol Diagnostics 1-Hourly\2020\MERRA2_400.tavg1_2d_aer_Nx.'+i+'.nc4')
    except:
        data =Dataset(r'J:\MERRA-2 Aerosol Diagnostics 1-Hourly\2020\MERRA2_401.tavg1_2d_aer_Nx.'+i+'.nc4')
    dstsurmas  = np.array(data['DUSMASS25'][:,:,:])
    orgcsurmas = np.array(data['OCSMASS'][:,:,:])
    bcsurmas   = np.array(data['BCSMASS'][:,:,:])
    sssurmas   = np.array(data['SSSMASS25'][:,:,:])
    so4surmas  = np.array(data['SO4SMASS'][:,:,:])
    
    dstsurmas_ave  =np.mean(dstsurmas, axis = 0)
    orgcsurmas_ave =np.mean(orgcsurmas, axis = 0)
    bcsurmas_ave   =np.mean(bcsurmas, axis = 0)
    sssurmas_ave   =np.mean(sssurmas, axis = 0)
    so4surmas_ave  =np.mean(so4surmas, axis = 0)
    
#     so4surf_ave = np.mean(so4surf, axis = 0)
    DUSMASS25_con.append(dstsurmas_ave)
    OCSMASS_con.append(orgcsurmas_ave)
    BCSMASS_con.append(bcsurmas_ave)
    SSSMASS25_con.append(sssurmas_ave)
    SO4SMASS_con.append(so4surmas_ave)

DUSMASS25 = np.array(DUSMASS25_con)
OCSMASS   = np.array(OCSMASS_con)
BCSMASS   = np.array(BCSMASS_con)
SSSMASS25 = np.array(SSSMASS25_con)
SO4SMASS  = np.array(SO4SMASS_con)
lat = np.array(data['lat'])
lon = np.array(data['lon'])

In [ ]:
# DUSMASS25 + OCSMASS+ BCSMASS + SSSMASS25 + SO4SMASS* (132.14/96.06)
DUSMASS25_abbreviation = 'Dust Surface Mass Concentration - PM 2.5'
OCSMASS_abbreviation = 'Organic Carbon Surface Mass Concentration __ENSEMBLE__'
BCSMASS_abbreviation = 'Black Carbon Surface Mass Concentration'
SSSMASS25_abbreviation = 'Sea Salt Surface Mass Concentration - PM 2.5'
SO4SMASS_abbreviation = 'SO4 Surface Mass Concentration __ENSEMBLE__'
PM25_formula = 'PM2.5 = DUSMASS25 + OCSMASS+ BCSMASS + SSSMASS25 + SO4SMASS* (132.14/96.06)'

np.savez('MERRA2_2020_PM25_Variables', DUSMASS25=DUSMASS25,OCSMASS=OCSMASS,BCSMASS=BCSMASS,SSSMASS25=SSSMASS25,
         SO4SMASS=SO4SMASS,lat=lat,lon=lon,dates=dates,DUSMASS25_abbreviation=DUSMASS25_abbreviation,
        OCSMASS_abbreviation=OCSMASS_abbreviation,BCSMASS_abbreviation=BCSMASS_abbreviation,
         SSSMASS25_abbreviation=SSSMASS25_abbreviation,SO4SMASS_abbreviation=SO4SMASS_abbreviation,PM25_formula=PM25_formula)

In [ ]:
npzfile=np.load('MERRA2_2018_PM25_Variables.npz') # Open .npz file
sorted(npzfile.files)

In [ ]:
# DUSMASS25 + OCSMASS+ BCSMASS + SSSMASS25 + SO4SMASS* (132.14/96.06)
#[PM2.5] = 1.375 × [SO4] + 1.8 × [OC] + [BC] + [DU2.5] +[SS2.5]
# pm25 = npzfile['DUSMASS25_con']+npzfile['OCSMASS_con']+npzfile['BCSMASS_con']+npzfile['SSSMASS25_con']+npzfile['SO4SMASS_con']*(132.14/96.06)
pm25 = (1.375*npzfile['SO4SMASS_con'])+ (1.8*npzfile['OCSMASS_con'])+npzfile['BCSMASS_con']+npzfile['DUSMASS25_con']+npzfile['SSSMASS25_con']

In [ ]:
pm25 = (1.375*SO4SMASS)+ (1.8*OCSMASS)+BCSMASS_con+DUSMASS25_con+SSSMASS25_con

In [ ]:
pm25_normalized = pm25*100000000

In [ ]:
pm25_normalized.shape

In [ ]:
dates = np.array(npzfile['dates'],dtype = 'str')
lon = np.array(npzfile['lon'],dtype = 'int')
lat = np.array(npzfile['lat'],dtype = 'int')

In [ ]:
a = np.arange(0,366)
for i in a:
    for j in dates:
        plt.figure(figsize=(14,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-180,179,-90,90],crs=ccrs.PlateCarree())
        plt.contourf(lon, lat, pm25_normalized[i,:,:], 60,transform=ccrs.PlateCarree())
        c_scheme=plt.pcolor(lon, lat, pm25_normalized[0,:,:],vmin = 1,vmax=100,cmap='gist_ncar')
        ax.gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False)
        cbar=plt.colorbar(c_scheme,orientation="horizontal")
        ax.coastlines()
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='black', alpha=0.5, linestyle='dotted')
        gl.xlabels_top = True # False real
        gl.xlabels_bottom = True # Not exixting
        gl.ylabels_left = True
        gl.ylabels_right = True
        gl.xlines = True
#         gl.xlocator = mticker.FixedLocator([40,45,50,55,60,65,70,75,80,85,90])
#         gl.ylocator = mticker.FixedLocator([-15,-10,-5,0,5,10,15,20,25,30])
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 10,'color': 'red', 'weight': 'bold'}
        gl.ylabel_style = {'size': 10,'color': 'red', 'weight': 'bold'}
        cbar.set_label('PM 2.5', rotation=0,weight= 'bold', size = 10)
        plt.savefig(j+'.jpg',dpi = 720)

In [ ]:
# Not a part of MERRA2 Data file creation
import glob
import os

os.chdir(r'J:\MERRA-2 Aerosol Diagnostics 1-Hourly\2016')
myFiles = glob.glob('*.nc4')
# print(myFiles)

In [ ]:
a = np.arange(0,366)
for i in a:
    plt.figure(figsize=(14,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-180,179,-90,90],crs=ccrs.PlateCarree())
    plt.contourf(lon, lat, pm25_normalized[i,:,:], 60,transform=ccrs.PlateCarree())
    c_scheme=plt.pcolor(lon, lat, pm25_normalized[i,:,:],vmin = 1,vmax=100,cmap='gist_ncar')
    ax.gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False)
    cbar=plt.colorbar(c_scheme,orientation="horizontal")
    ax.coastlines()
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='black', alpha=0.5, linestyle='dotted')
    gl.xlabels_top = True # False real
    gl.xlabels_bottom = True # Not exixting
    gl.ylabels_left = True
    gl.ylabels_right = True
    gl.xlines = True
    #gl.xlocator = mticker.FixedLocator([40,45,50,55,60,65,70,75,80,85,90])
    #gl.ylocator = mticker.FixedLocator([-15,-10,-5,0,5,10,15,20,25,30])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 10,'color': 'red', 'weight': 'bold'}
    gl.ylabel_style = {'size': 10,'color': 'red', 'weight': 'bold'}
    cbar.set_label('PM 2.5', rotation=0,weight= 'bold', size = 20)
    plt.savefig(dates[i]+'.jpg',dpi = 720)

# MODIS AOD Data visualization

In [ ]:

# Python program to demonstrate
# HDF5 file
 
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pyhdf.SD import SD, SDC

In [ ]:
# Open file.
path = r'D:\Downloads\MOD04_L2\2018001'
file_name = path + '\MOD04_L2.A2018001.0000.061.2018003202444.hdf'

file = SD(file_name, SDC.READ)

print(file)

In [ ]:
# Display all the datasets
datasets_dic = file.datasets()

for idx,sds in enumerate(datasets_dic.keys()):
    print(idx,sds)

In [ ]:
lat_obj = file.select('Latitude') # select sds
lon_obj = file.select('Longitude') # select sds 
lon_AOD_550_Dark_Target = file.select('AOD_550_Dark_Target_Deep_Blue_Combined')

lat     = lat_obj.get() # get sds data
lon     = lon_obj.get() # get sds data
AOD_550 = lon_AOD_550_Dark_Target.get()


In [ ]:
AOD_550_good = np.where(AOD_550 == -9999, np.nan, AOD_550)

In [ ]:
plt.contourf(lon, lat, AOD_550_good, 60,cmap = 'jet')

In [ ]:
lat.max()

In [ ]:
dates